<h3 style="background-color: #D3D92B;"><br>Numerical Optimisation. Project 1<br></h3><br>

<h3 style="background-color: #D3D92B;"><br>Team Information<br></h3><br>

<i>Group 7<br>
Participants information in alphabetical order</i>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Alexander</td>
    <td style = "text-align: left">Mair</td>
    <td style = "text-align: left">k11916624</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Andreas</td>
    <td style = "text-align: left">Oberdammer</td>
    <td style = "text-align: left">k11908776</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Dominik</td>
    <td style = "text-align: left">Zauner</td>
    <td style = "text-align: left">k11717988</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Florian</td>
    <td style = "text-align: left">Rothkegel</td>
    <td style = "text-align: left">k11908775</td>
  </tr>
  <tr>
    <td style = "text-align: left">5</td>
    <td style = "text-align: left">Martin</td>
    <td style = "text-align: left">Stockinger</td>
    <td style = "text-align: left">k01035089</td>
  </tr>
  <tr>
    <td style = "text-align: left">6</td>
    <td style = "text-align: left">Martin</td>
    <td style = "text-align: left">Zwiffl</td>
    <td style = "text-align: left">k11910668</td>
  </tr>

</table>


<h3 style="background-color: #D3D92B;"><br>Implementation<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Imports<br></h4><br>
<i>Describe how to install additional packages, if you have some, here</i><br><br>

<b>No additional libaries are needed/installed</b>



In [1]:
import numpy as np
import scipy 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

<hr><h4 style="background-color: #ADB8FF;"><br>Stopping criteria<br></h4><br>
<i>Place for additional comments and argumentation</i><br><br>
We implemented the stopping criterion based on the hints given in the second phase PDF. Description of the funciton can be found in the code

In [36]:
#your function for stopping criterium
def stopping_criteria(x_start, x_curr, x_prev, x_after_start, fn, gradient_fn, iteration, tolerance=1e-7, max_steps=1e6):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_start: Start point coordinates. Postion on which the algorithm should start
    --------------------------------------------------------------------------------------------------------------
    x_curr: Current point coordinates. Postion on which the algorithm is currently
    --------------------------------------------------------------------------------------------------------------
    x_prev: Previous point coordinates. Postion on which the algorithm is was in the step before
    --------------------------------------------------------------------------------------------------------------
    x_after_start: Fist calculated point coordinates. Postion on which the algorithm was after the first step
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    iteration: Number of the current iteration the algorithm is in. Starts with 0. Is used to check max_steps
    --------------------------------------------------------------------------------------------------------------
    tolerance: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    max_steps: Number of the current iteration the algorithm is in. Starts with 0. Is used to check max_steps
    --------------------------------------------------------------------------------------------------------------
    - this function checks if the gradient of the current point is below the tolerance
    - Additionally it checks if the algo exceeds the allowed max steps
    - It is also checked, if the gradient of the current step is smaller than the gradient in the start point
      times our tolerance. In our case we noticed by testing, that the algorithm stops a bit too soon, therefore
      we introduced a correction factor, which additionally lowers the error bound. Again --> Payoff runtime/acc
    - As mentioned in the second phase pdf, we can make the stopping crit relative, by comparing it to the 
      tolerance multiplied with the difference from the start to the first point. this can be done for f(x) and x.
    """
    correction_factor = 1e-5 # factor to adjust error bound and gain more precicsion (later stopping). Explained above.
    
    if np.linalg.norm(gradient_fn(x_curr)) <= tolerance:
        print("*** GRADIENT BELOW TOLERANCE ***")
        return True
    
    if iteration >= max_steps:
        print("*** MAX STEPS EXCEEDED ***")
        return True
    
    if np.linalg.norm(gradient_fn(x_curr)) <= tolerance * np.linalg.norm(gradient_fn(x_start)) * correction_factor:
        print("*** GRADIENT SMALLER THAN GRADIENT OF FIRST STEP * TOLERANCE ***")
        return True
    
    if x_prev is not None and x_after_start is not None:
        x_diff_first_step = np.linalg.norm(x_after_start - x_start)
        y_diff_first_step = np.abs(fn(x_after_start) - fn(x_start))

        if np.abs(fn(x_curr) - fn(x_prev)) <= tolerance * y_diff_first_step * correction_factor:
            print("*** Y DIFFERENCE SMALLER THAN ADJUSTED TOLERANCE ***")
            return True
        
        if np.linalg.norm(x_curr - x_prev) <= tolerance * x_diff_first_step * correction_factor:
            print("*** X DIFFERENCE SMALLER THAN ADJUSTED TOLERANCE ***")
            return True
    

<hr><h4 style="background-color: #ADB8FF;"><br>Varibales scaling<br></h4><br>
<i>Place your reasoning here, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.<i>

<h1>Scaling</h1>
    
Scaling is a major topic when it comes to improving the performance and stability of an optimization algorithm. Generally, it can be viewed as harmonizing the magnitudes of variables. Scaling does not seem to have a pinpoint definition in literature but at the same time seems to be commonly understood. In our examples, we want to take a look at scaling by variable transformation:
    
<i>"Scaling by variable transformation converts the variables from units that typically reflect the physical nature of the problem to units that display certain desirable properties during the minimization process."</i> - Gill, Murray, Wright (1981): Practical Optimization

Also, in some online resources we find an explanation of : <i>"Scale x so its components are “around 1”"</i> <a href="http://www.fitzgibbon.ie/optimization-parameter-scaling">LINK</a>

Also, it should be noted that some algorithms are sensible to scaledness/unscaledness of a problem. One of those is Steepest Descent/Gradient Descent. Therefore we will take a look at our Gradient Descent implementation regarding scaling.

To line out the basic idea more practically, consider the following simple objective functions that should be minimized:

<ul> 
    <li>$f(x,y) = 1000x^2+10y^2$</li>
    <li>$f(x,y) = 10^6x^2+10^6y^2$</li> 
</ul>

The first function can be scaled by a diagonal matrix $D=\begin{bmatrix}\frac{1}{1000} & 0 \\ 0 & \frac{1}{10} \end{bmatrix}$

The second function can be viwed as scaled by just dividing it straightforward by $10^6$.

But just scaling the the inputs does not really affect the inner workings of our algorithms. We make use of the logic shown in our example scaled Gradient Descent below.

<h3>Scaling and our problems</h3>

Generally, since the entries of our base matrices A_x for our multivariate Problems range only between 0 and 9, the influences of single variables are somewhat of similar magnitude. Simplifiying this thought by stating that the term $x^TAx$ comes out the polynomials with factors of magnitude 10 - 100, we argue that we can scale down the variables just by $\frac{1}{10}$ to get closer to the goal of <i>"scaling x to, so its components are around 1"</i>. So we do not use scaling diagonal matrix but simply a factor (can be seen as a diagonal matrix with all the same values).

In a nutshell, scaling should not be super-crucial for our somewhat nicely scaled problems. As we will see later, we can still save some iterations on our implementation. If we had many more strongly diverging magnitudes of variable impact, it can become a key point for success!

In [30]:
def problem_sclaing(xk, gradient_fn, scaling, activate_new_stop_crit, epsilon, fn):
        """

        Parameters
        --------------------------------------------------------------------------------------------------------------
        xk: Start point coordinates. Postion on which the algorithm should start
        --------------------------------------------------------------------------------------------------------------
        fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
        --------------------------------------------------------------------------------------------------------------
        gradient_fn: function, which is the gradient of the problem. 
                     One can parse coordinates to this function to get the gradient.
        --------------------------------------------------------------------------------------------------------------
        scaling: Value, which defines the strength of the scaling algorithm. > Stronger scaling.
        --------------------------------------------------------------------------------------------------------------
        epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
                 algorithm --> basic stopping criterion
        --------------------------------------------------------------------------------------------------------------
        activate_new_stop_crit: Boolean Flag, which de/activates the usage of the additinal stopping criterations.
                                If activated, the implemented stopping crit function is called and all features
                                get checked.
        --------------------------------------------------------------------------------------------------------------
        - Scale xk with the scaling factor in every iteration to get yk
        - Evaluate gradient/objective on initial unscaled xk
        - Get step size
        - Then operate on the scaled yk
        - The result needs to be rescaled upon returning
        """
        x_start = xk
        yk = scaling * xk
        prev_x = None
        x_step_1 = None
        i = 0
        while True:
            prev_x = xk
            xk = yk / scaling
            
            if i == 1:
                x_step_1 = xk
            
            grad = gradient_fn(xk) * scaling
            grad_norm = np.linalg.norm(grad / scaling)

            if activate_new_stop_crit and stopping_criteria(x_start, xk, prev_x, x_step_1, fn, gradient_fn, i):
                break

            elif not activate_new_stop_crit and grad_norm <= epsilon:
                break

            alpha = backtracking_line_search(xk, fn, gradient_fn, alpha=1, phi=0.9, c=0.0001)
            yk = yk - (alpha * grad)
            i += 1

        print("performed " + str(i+1) + " iterations.")

        return yk / scaling

<hr><h4 style="background-color: #ADB8FF;"><br>Stabilising algorithm<br></h4><br>
<i>Place your reasoning here, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.</i>
<br><br>
Stability is a really important topic for those type of algorithms, because if the stability is not given, it is not ensured that we even reach our goal, because the algorithm could do basically anything and even crash.<br>
Therefore it is really important to take some measures to ensure that the algorithm is stable and works fine.<br><br>

Possible Measures:<br>
- Avoid floating points problems and round off errors<br>
- Scale problems if necessary<br>
- Avoid matrix inversion<br>
- Use optimal hyperparamters for algorithm<br>
- Improve stopping criterion (limit max steps, verify multiple attributes)<br><br>

We implemented some of those measure. We tuned our tolerance (epsilon) and the step size to avoid overshooting of the algorithm. We also use float calculation to avoid big round off errors (read more in floating point section).<br>
We also use LU decomposition to avoid direct matrix inversion and we also improved our stopping criterion. These measure have a positive impact on the stability of our algorithm. <br>
<br>
There are for sure more stability improvements, but as we can see on our tests, the algorithm is performing reasonable well and therefore we did not implement more features. <br><br>

<b>Below we want to show a described problem, if we would not use default float arrays for our calculation.</b><br>
You should be able to see that the numpy result (which is correct result) is equal to the result, if we typecast the int array to an float array. If we just use the int array, we get wrong results and this destroys the stability of the aglo and makes it nearly unuseable.
<br><b>Therefore is the useage of float a major contribution for stability </b>

In [4]:
#your function for stabilising goes here
def lu_deco_inverse_not_float(A):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    A: Input matrix, from which the inverse has to be calculated 
    --------------------------------------------------------------------------------------------------------------
    - LU Decompostion matrix factorization, creates two trinangular matrices L and U. In this case this is a bit
      simpler. We create a permuation matrix and a matrix (a), which is a combination of L and U.
    - Based on this matrices we are able to create the inverted version of the given matrix
    - Float conversion is missing in this function to show the problems, which happen with no typecasting and int
      calculation
    """
    n = A.shape[0]
    a = np.copy(A)
    p = np.eye(n)

    for i in range(n - 1):
        j = i + np.argmax(np.abs(a[i:, i]))  # maximum in column
        if np.abs(a[i, j]) < 1e-8:
            print("Matrix is singular")
            sys.exit()

        if i != j:
            a[[i, j]] = a[[j, i]]
            p[[i, j]] = p[[j, i]]

        for i_row in range(i + 1, n):
            a[i_row, i] = a[i_row, i] / a[i, i]
        for i_row in range(i + 1, n):
            for i_col in range(i + 1, n):
                a[i_row, i_col] = a[i_row, i_col] - np.outer(a[i_row, i], a[i, i_col])

    a_inv = np.copy(np.dot(p, np.eye(n)))

    for i in range(n):
        for j in range(i):
            a_inv[i] -= np.dot(a[i, j], a_inv[j])  # Forward substitution

    for i in reversed(range(n)):
        for j in range(i + 1, n):
            a_inv[i] = a_inv[i] - np.dot(a[i, j], a_inv[j])  # Backward substitution
        a_inv[i] = a_inv[i] / a[i, i]

    return a_inv

def lu_deco_inverse_float(A):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    A: Input matrix, from which the inverse has to be calculated 
    --------------------------------------------------------------------------------------------------------------
    - LU Decompostion matrix factorization, creates two trinangular matrices L and U. In this case this is a bit
      simpler. We create a permuation matrix and a matrix (a), which is a combination of L and U.
    - Based on this matrices we are able to create the inverted version of the given matrix
    - Float conversion is done here to get the correct results (no round of error/auto typecasting problem)
    """
    n = A.shape[0]
    a = np.copy(A.astype(float))
    p = np.eye(n)

    for i in range(n - 1):
        j = i + np.argmax(np.abs(a[i:, i]))  # maximum in column
        if np.abs(a[i, j]) < 1e-8:
            print("Matrix is singular")
            sys.exit()

        if i != j:
            a[[i, j]] = a[[j, i]]
            p[[i, j]] = p[[j, i]]

        for i_row in range(i + 1, n):
            a[i_row, i] = a[i_row, i] / a[i, i]
        for i_row in range(i + 1, n):
            for i_col in range(i + 1, n):
                a[i_row, i_col] = a[i_row, i_col] - np.outer(a[i_row, i], a[i, i_col])

    a_inv = np.copy(np.dot(p, np.eye(n)))

    for i in range(n):
        for j in range(i):
            a_inv[i] -= np.dot(a[i, j], a_inv[j])  # Forward substitution

    for i in reversed(range(n)):
        for j in range(i + 1, n):
            a_inv[i] = a_inv[i] - np.dot(a[i, j], a_inv[j])  # Backward substitution
        a_inv[i] = a_inv[i] / a[i, i]

    return a_inv

test_mat = np.array([[2,-1,0], [1,2,-2], [0,-1,1]]) # matrix to test scenario on. Can be chosen abitrarly

print("============================================================================")
print("MATRIX INVERSION WITH AND WITHOUT FLOAT CALCULATION")
print("============================================================================")
print("INVERSE MATRIX WITHOUT PRE FLOAT TYPECASTING")
print(lu_deco_inverse_not_float(test_mat))
print("============================================================================")
print("INVERSE MATRIX WITH PRE FLOAT TYPECASTING")
print(lu_deco_inverse_float(test_mat))
print("============================================================================")
print("INVERSE MATRIX NUMPY CALCULATION (TRUE RESULT)")
print(np.linalg.inv(test_mat))


MATRIX INVERSION WITH AND WITHOUT FLOAT CALCULATION
INVERSE MATRIX WITHOUT PRE FLOAT TYPECASTING
[[0.5  0.25 0.5 ]
 [0.   0.5  1.  ]
 [0.   0.   1.  ]]
INVERSE MATRIX WITH PRE FLOAT TYPECASTING
[[-1.11022302e-16  1.00000000e+00  2.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  4.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  5.00000000e+00]]
INVERSE MATRIX NUMPY CALCULATION (TRUE RESULT)
[[-1.11022302e-16  1.00000000e+00  2.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  4.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  5.00000000e+00]]


<hr><h4 style="background-color: #ADB8FF;"><br>Fighting floating-point numbers and roundoff error<br></h4><br>
<i>Place your reasoning, how your algorithm behave with respect to this problem. You can also try rescaling your problems
This is additional task, which can earn you several points.</i>
<br><br>
In our algorithm and calculations we only use floating point numbers. There is always the risk of roundoff errors and therefore loss in accuracy, which leads to increased error rates.<br>
We are fully aware of this problem and therefore suggest the following options:<br>
- Do every calculation in float, in order to avoid typecasting and high round of errors<br>
- Increase precision of float (assign more memory (bits)) to the variables and reduce error (increased memory usage + computaton time)<br>
- We could also use a special python libary for high precision calculation. Same as before, this increases memory usage and computation time.<br><br>

As we can see, there are some measure we can take, in order to avoid round off errors. The most important measure is the first one. If we calculate everything in float, the round off error is small enough <br>
and as we can see, based on our tests below, we do not have any problems, therefore this measure seems to be enough. If we really have the need for super high precision calculation we have to either increase the<br>
memory size of float or use an external libary

<hr><h4 style="background-color: #ADB8FF;"><br>Inverting matrices<br></h4><br>
<i>Place for additional comments and argumentation</i><br><br>
    
The algorithm of choice for inverting matrices is based on the approach from the book <b> Numerical Optimization</b> written by <b><i> Jorge Nocedal, Stephen J. Wright</i></b> and the calculator of TU Graz (http://lampx.tugraz.at/~hadley/num/ch2/2.3.php).<br>
We do not directly decompose the matrix into L and U, because this not really needed in our case, since we reuse the LU matrix and do not use the two parts separate. <i>a_inv</i> is basically the the LU matrix, which is than used to create inverse.
<br><br><b>IMPORTANT:</b> This is also mentioned in the floating point/error section. If the matrix is parsed with int values, python has to do typecasting, because of the divison, which is done in this function. As far as our testing goes, typecasting leads<br>
to an major error in calculation and leads to wrong inverse matrices. This is not only the case for this function, but also in some other cases we encountered this problem. For more details check the floating point/stability section

In [5]:
#your function for invertion goes here
def lu_deco_inverse(A):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    iteration: Number of the current iteration the algorithm is in. Starts with 0. Is used to check max_steps
    --------------------------------------------------------------------------------------------------------------
    tolerance: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    max_steps: Number of the current iteration the algorithm is in. Starts with 0. Is used to check max_steps
    --------------------------------------------------------------------------------------------------------------
    - this function checks if the gradient of the current point is below the tolerance
    - Additionally it checks if the algo exceeds the allowed max steps
    - It is also checked, if the gradient of the current step is smaller than the gradient in the start point
      times our tolerance. In our case we noticed by testing, that the algorithm stops a bit too soon, therefore
      we introduced a correction factor, which additionally lowers the error bound. Again --> Payoff runtime/acc
    - As mentioned in the second phase pdf, we can make the stopping crit relative, by comparing it to the 
      tolerance multiplied with the difference from the start to the first point. this can be done for f(x) and x.

    --------------------------------------------------------------------------------------------------------------
    - LU Decompostion matrix factorization, creates two trinangular matrices L and U. In this case this is a bit
      simpler. We create a permuation matrix and a matrix (a), which is a combination of L and U.
    - Based on this matrices we are able to create the inverted version of the given matrix
    - Float conversion is done here to get the correct results (no round of error/auto typecasting problem)
    """
    n = A.shape[0]
    a = np.copy(A.astype(float))
    p = np.eye(n)

    for i in range(n - 1):
        j = i + np.argmax(np.abs(a[i:, i]))  # maximum in column
        if np.abs(a[i, j]) < 1e-8:
            print("Matrix is singular")
            sys.exit()

        if i != j:
            a[[i, j]] = a[[j, i]]
            p[[i, j]] = p[[j, i]]

        for i_row in range(i + 1, n):
            a[i_row, i] = a[i_row, i] / a[i, i]
        for i_row in range(i + 1, n):
            for i_col in range(i + 1, n):
                a[i_row, i_col] = a[i_row, i_col] - np.outer(a[i_row, i], a[i, i_col])

    a_inv = np.copy(np.dot(p, np.eye(n)))

    for i in range(n):
        for j in range(i):
            a_inv[i] -= np.dot(a[i, j], a_inv[j])  # Forward substitution

    for i in reversed(range(n)):
        for j in range(i + 1, n):
            a_inv[i] = a_inv[i] - np.dot(a[i, j], a_inv[j])  # Backward substitution
        a_inv[i] = a_inv[i] / a[i, i]

    return a_inv

<hr><h4 style="background-color: #ADB8FF;"><br>Gradients calculation<br></h4><br>
<i>Place for additional comments and argumentation</i>
<br><br>  
As suggested, we now approximate derivative and hessians. We did this by using the trivial formulas of section 8.1 (central- difference). <br>
4 wrapper functions were created, because the different shapes, so we can use the aproximated derivatives, gradients, second_derivatives and hessians just as before.

In [6]:
#your function for gradient approximation goes here
def differential_quotient_helper(fn, x, epsilon=1e-9):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    x: point, which should be taken into account. Mostly used for the shape to recreate correct der/grad shape
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Algorithm, which approximates numericaly the gradient of a given function f
    """
    res=[]
    x = x.astype(float)
    x=x.reshape(x.shape[1] if len(x.shape) > 1 and x.shape[1] > x.shape[0] else x.shape[0])
    for idx in range(len(x)):
        direction = np.zeros(len(x))
        direction[idx] = 1
        part_derivative = (fn(x+epsilon*direction)-fn(x-epsilon*direction))/(2*epsilon)
        res.append(part_derivative)
    return np.array(res)

def grad_wrapper(fn,  epsilon=1e-4):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Wrapper, which returns the useable gradient function of a given problem function fn
    """
    return lambda x: differential_quotient_helper(fn, x,  epsilon)

def hessian_wrapper(fn, epsilon=1e-4):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Wrapper, which returns the useable hessian function of a given problem function fn
    """
    return lambda x: grad_wrapper(grad_wrapper(fn, epsilon), epsilon)(x).reshape(len(x),len(x))

def derivative_wrapper(fn, epsilon=1e-4):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Wrapper, which returns the useable derivaitve function of a given problem function fn
    """
    return lambda x: (fn(x+epsilon)-fn(x-epsilon))/(2*epsilon)

def second_derivative_wrapper(fn, epsilon=1e-4):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    - Wrapper, which returns the useable second derivative function of a given problem function fn
    """
    return lambda x: derivative_wrapper(derivative_wrapper(fn, epsilon), epsilon)(x)

<hr><h4 style="background-color: #ADB8FF;"><br>Additional objects you implemented<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [7]:
#your code goes here
def backtracking_line_search(x, fn, gradient_fn, alpha, phi, c):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x: Start point coordinates. Postion on which the algorithm should start
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    alpha - phi - c: Line search specific hyperparameters
    --------------------------------------------------------------------------------------------------------------
    - backtracking line search used for steepest descent
    """
    pk = - gradient_fn(x)
    left = fn(x + alpha * pk)
    right = fn(x) + c * alpha * (gradient_fn(x).reshape(-1) @ pk.reshape(-1))
    while left > right:
        alpha = alpha * phi
        left = fn(x + alpha * pk)
        right = fn(x) + c * alpha * (gradient_fn(x).reshape(-1) @ pk.reshape(-1))
    return alpha

<hr><h4 style="background-color: #ADB8FF;"><br>Optimising algorithm itself<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [8]:
#your code goes here
# Our Main SD algorithm for multivariate functions
def steepest_descent_multi(x_start, fn, gradient_fn, epsilon=0.8e-5,
         activate_new_stop_crit=False,
         activate_pre_scaling=False,
         activate_alt_gradient=False,
         activate_new_matrix_inversion=False,
         scaling=None):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_start: Start point coordinates. Postion on which the algorithm should start
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    activate_new_stop_crit: Boolean Flag, which de/activates the usage of the additinal stopping criterations.
                            If activated, the implemented stopping crit function is called and all features
                            get checked.
    --------------------------------------------------------------------------------------------------------------
    activate_pre_scaling: Boolean Flag, to activate pre scaling of the problem. In this case, we have not 
                          implemented pre scaling.
    --------------------------------------------------------------------------------------------------------------
    activate_alt_gradient: Boolean Flag, to activate alternative gradient calculation. This calls the gradient
                           wrapper and approximates the gradient function.
    --------------------------------------------------------------------------------------------------------------
    activate_new_matrix_inversion: Boolean Flag, to activate alternative matrix inversion. This calls the LU
                                   decomposition function, but is not needed in this algo since nothing needs to
                                   be inversed.
    --------------------------------------------------------------------------------------------------------------
    - steepest descent method for quadractic multi variate examples
    """
    if activate_alt_gradient:
        gradient_fn = grad_wrapper(fn)
    
    if not activate_pre_scaling:
        i = 0
        xk = x_start
        prev_x = None
        x_step_1 = None 
        while True:  
            grad = gradient_fn(xk)
            grad_norm = np.linalg.norm(grad)

            if not activate_new_stop_crit and grad_norm <= epsilon:
                break

            elif activate_new_stop_crit and stopping_criteria(x_start, xk, prev_x, x_step_1, fn, gradient_fn, i):
                break

            alpha = backtracking_line_search(xk, fn, gradient_fn, alpha=1, phi=0.9, c=0.0001)
            prev_x = xk
            xk = xk - (alpha * grad)
            
            if i == 0:
                x_step_1 = xk
                
            i += 1

        print("performed " + str(i+1) + " iterations.")

        return xk

    elif activate_pre_scaling:
        if scaling is None:
            raise ValueError('You have to choose a value for *scaling*')
        return  problem_sclaing(x_start, gradient_fn, scaling, activate_new_stop_crit, epsilon, fn)
    


# Our Main SD algorithm for univariate functions
def steepest_descent_uni(x_start, fn, derivative, lr=1e-7, epsilon=1e-5,
         activate_new_stop_crit=False,
         activate_pre_scaling=False,
         activate_alt_gradient=False,
         activate_new_matrix_inversion=False):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_start: Start point coordinates. Postion on which the algorithm should start
    --------------------------------------------------------------------------------------------------------------
    fn: function, which is the problem. One can parse coordinates to this function to get y-axis values
    --------------------------------------------------------------------------------------------------------------
    gradient_fn: function, which is the gradient of the problem. 
                 One can parse coordinates to this function to get the gradient.  
    --------------------------------------------------------------------------------------------------------------
    lr: learning rate, which defines the size of the steps the algorithm can make per iteration. 
        Definese the aggresivity of the algorithm, can increase/decrease overshoot.
    --------------------------------------------------------------------------------------------------------------
    epsilon: tolerance of the algorithm. If this values is reached and our values are below it, we can stop the
             algorithm --> basic stopping criterion
    --------------------------------------------------------------------------------------------------------------
    activate_new_stop_crit: Boolean Flag, which de/activates the usage of the additinal stopping criterations.
                            If activated, the implemented stopping crit function is called and all features
                            get checked.
    --------------------------------------------------------------------------------------------------------------
    activate_pre_scaling: Boolean Flag, to activate pre scaling of the problem. In this case, we have not 
                          implemented pre scaling.
    --------------------------------------------------------------------------------------------------------------
    activate_alt_gradient: Boolean Flag, to activate alternative gradient calculation. This calls the gradient
                           wrapper and approximates the gradient function.
    --------------------------------------------------------------------------------------------------------------
    activate_new_matrix_inversion: Boolean Flag, to activate alternative matrix inversion. This calls the LU
                                   decomposition function, but is not needed in this algo since nothing needs to
                                   be inversed.
    --------------------------------------------------------------------------------------------------------------
    - steepest descent method for above quadratic uni variate examples
    """
    if activate_alt_gradient:
        derivative = derivative_wrapper(fn)
    
    xk = x_start
    prev_x = None
    x_step_1 = None
    i = 0
    while True:
        der = derivative(xk)
        
        if i == 1:
            x_step_1 = xk

        if activate_new_stop_crit and stopping_criteria(x_start, xk, prev_x, x_step_1, fn, derivative, i):
            break

        elif not activate_new_stop_crit and abs(der) <= epsilon:
            break
        
        prev_x = xk
        xk=xk-(lr * der)
        i += 1
    print("performed " + str(i+1) + " iterations.")
    return xk

<h3 style="background-color: #D3D92B;"><br>Testing on 5-10 variables, Quadratic objective<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Implement functions to optimise over<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [9]:
#your code goes here
def get_random_A_b_minimizer(seed):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    seed: Seed is an int value, which is used to create reproducable random values.
    --------------------------------------------------------------------------------------------------------------
    - 10x10 random matrix is produced by this function, which is the base of the problem
    - 10x1 random vector is produced, which is also the minimizer of the function and the goal of the algo
    - Function uses thoes variables to create a problem function, which can be solved by the algo
    """
    np.random.seed(seed)
    A=np.random.randint(0, 2, (10, 10))
    A=A@A.T

    np.random.seed(seed)
    minimizer=np.random.randint(0, 10, (10, 1))

    b=A@minimizer

    return A, b, minimizer

# Created problem instances, which are used in the functions below, to provide a useable set of prolems for the algo, A and b correspond to function depended values and Minimizer is the solution to the problem
A_1, b_1, minimizer_1 = get_random_A_b_minimizer(0)
A_2, b_2, minimizer_2 = get_random_A_b_minimizer(2)
A_3, b_3, minimizer_3 = get_random_A_b_minimizer(3)
A_4, b_4, minimizer_4 = get_random_A_b_minimizer(12)
A_5, b_5, minimizer_5 = get_random_A_b_minimizer(15)

#############################################################################################################################################################
# problem_x_q(x) --> is the problem function f, which is used by algorithm.
# grad_problem_X_q(x) --> as the name implies, this function is the gradient of the problem function f (problem_X_q)
# shessian_problem_X_q(x) --> as the name implies, this function is the hessian of the problem function f (problem_X)
# in all cases you can parse x coordinates to the functions in order to receive the corresponding y axis values.
#############################################################################################################################################################
# Problem 1

def problem_1_q(x):
    return (x.T@A_1@x)/2-b_1.T@x

def grad_problem_1_q(x):
    return np.dot(A_1,x)-b_1

def hessian_problem_1_q(x):
    return A_1

#############################################################################################################################################################
# Problem 2

def problem_2_q(x):
    return (x.T@A_2@x)/2-b_2.T@x

def grad_problem_2_q(x):
    return A_2@x-b_2

def hessian_problem_2_q(x):
    return A_2

#############################################################################################################################################################
# Problem 3

def problem_3_q(x):
    return (x.T@A_3@x)/2-b_3.T@x

def grad_problem_3_q(x):
    return A_3@x-b_3

def hessian_problem_3_q(x):
    return A_3

#############################################################################################################################################################
# Problem 4

def problem_4_q(x):
    return (x.T@A_4@x)/2-b_4.T@x

def grad_problem_4_q(x):
    return A_4@x-b_4

def hessian_problem_4_q(x):
    return A_4

#############################################################################################################################################################
# Problem 5

def problem_5_q(x):
    return (x.T@A_5@x)/2-b_5.T@x

def grad_problem_5_q(x):
    return A_5@x-b_5

def hessian_problem_5_q(x):
    return A_5

<hr><h4 style="background-color: #ADB8FF;"><br>Run 5 tests<br></h4><br>
<p><b>Note:</b> After every test print out the resulsts. 
<br>For your convinience we implemented a function which will do it for you. Function can be used in case after running optimisation you return $x_{optimal}$, and if you have implemented your gradient approximation. Feel free to bring your adjustments.
<br> Additionaly print how many iterations your algotithm needed. You might also provide charts of your taste (if you want).
<p><i>Place for your additional comments and argumentation<i>

In [10]:
def final_printout(x_0,x_optimal,x_appr,f,grad,args,tolerance):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    x_0: numpy 1D array, corresponds to initial point
    x_optimal: numpy 1D array, corresponds to optimal point, which you know, or have solved analytically
    x_appr: numpy 1D array, corresponds to approximated point, which your algorithm returned
    --------------------------------------------------------------------------------------------------------------
    f: function which takes 2 inputs: x (initial, optimal, or approximated)
                                      **args
       Function f returns a scalar output.
    --------------------------------------------------------------------------------------------------------------
    grad: function which takes 3 inputs: x (initial, optimal, or approximated), 
                                         function f,
                                         args (which are submitted, because you might need
                                              to call f(x,**args) inside your gradient function implementation). 
          Function grad approximates gradient at given point and returns a 1d np array.
    --------------------------------------------------------------------------------------------------------------
    args: dictionary, additional (except of x) arguments to function f
    tolerance: float number, absolute tolerance, precision to which, you compare optimal and approximated solution.
    """
    
    print(f'Initial x is :\t\t{x_0}')
    print(f'Optimal x is :\t\t{x_optimal}')
    print(f'Approximated x is :\t{x_appr}')
    print(f'Is close verificaion: \t{np.isclose(x_appr,x_optimal,atol=tolerance)}\n')
    f_opt = f(x_optimal)
    f_appr = f(x_appr)
    print(f'Function value in optimal point:\t{f_opt}')
    print(f'Function value in approximated point:   {f_appr}')
    print(f'Is close verificaion:\t{np.isclose(f_opt,f_appr,atol=tolerance)}\n')
    print(f'Gradient approximation in optimal point is:\n{grad(x_optimal)}\n')
    grad_appr = grad(x_appr)
    print(f'Gradient approximation in approximated point is:\n{grad_appr}\n')
    print(f'Is close verificaion:\n{np.isclose(grad_appr,np.zeros(grad_appr.shape),atol=tolerance)}')

In [11]:
#your code goes here
print("============================================================================")
print("DEFAULT TESTS WITHOUT ANY ADDITIONAL FEATURES")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
print("**** PROBLEM 1 ****")
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)
print("============================================================================")
print("**** PROBLEM 2 ****")
x_hat=steepest_descent_multi(init, problem_2_q, grad_problem_2_q)
final_printout(init, minimizer_2, x_hat, problem_2_q, grad_problem_2_q, {}, 1e-3)
print("============================================================================")
print("**** PROBLEM 3 ****")
x_hat=steepest_descent_multi(init, problem_3_q, grad_problem_3_q)
final_printout(init, minimizer_3, x_hat, problem_3_q, grad_problem_3_q, {}, 1e-3)
print("============================================================================")
print("**** PROBLEM 4 ****")
x_hat=steepest_descent_multi(init, problem_4_q, grad_problem_4_q)
final_printout(init, minimizer_4, x_hat, problem_4_q, grad_problem_4_q, {}, 1e-3)
print("============================================================================")
print("**** PROBLEM 5 ****")
x_hat=steepest_descent_multi(init, problem_5_q, grad_problem_5_q)
final_printout(init, minimizer_5, x_hat, problem_5_q, grad_problem_5_q, {}, 1e-3)

DEFAULT TESTS WITHOUT ANY ADDITIONAL FEATURES
**** PROBLEM 1 ****
performed 27426 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[ 4.99978826e+00]
 [-1.50251160e-04]
 [ 2.99993373e+00]
 [ 3.00033758e+00]
 [ 7.00007641e+00]
 [ 9.00028584e+00]
 [ 2.99966083e+00]
 [ 5.00008874e+00]
 [ 1.99979429e+00]
 [ 4.00005632e+00]]
Is close verificaion: 	[[False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[ 1.39131356e-06]
 [-2.35047878e-07]
 [ 1.60550849e-06]
 [ 3.51312744e-06]
 [ 2.50869056e-06]
 [ 3.16264908e-06]
 [-9.97649380e-07]
 [ 2.00431822e-06]
 [ 1.17500591e-06]


In [12]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED SCALING FOR PROBLEM 1")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q, scaling = 0.1, activate_pre_scaling=True)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)
print("============================================================================")
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q, scaling = 1000, activate_pre_scaling=True)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)
print("============================================================================")
print("VERSUS NO SCALING FOR PROBLEM 1")
print("============================================================================")
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q,  activate_pre_scaling=False)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED SCALING FOR PROBLEM 1
performed 27160 iterations. Final norm of gradient: 7.93894662732799e-06
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[ 4.99977126e+00]
 [-1.62317696e-04]
 [ 2.99992842e+00]
 [ 3.00036471e+00]
 [ 7.00008255e+00]
 [ 9.00030881e+00]
 [ 2.99963359e+00]
 [ 5.00009588e+00]
 [ 1.99977777e+00]
 [ 4.00006086e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[ 1.94171002e-06]
 [-1.57909788e-07]
 [ 2.07779323e-06]
 [ 4.10522560e-06]
 [ 3.08208456e-06]
 [ 3.71219630e-06]
 [-9.

In [13]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED ADVANCED STOPPING CRITERION FOR PROBLEM 1")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q, activate_new_stop_crit=True)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)
print("============================================================================")
print("VERSUS NORMAL STOPPING CRITERION FOR PROBLEM 1")
print("============================================================================")
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q, activate_new_stop_crit=False)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED ADVANCED STOPPING CRITERION FOR PROBLEM 1
*** Y DIFFERENCE SMALLER THAN ADJUSTED TOLERANCE ***
performed 18962 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[ 4.99753514e+00]
 [-1.74993767e-03]
 [ 2.99922937e+00]
 [ 3.00393383e+00]
 [ 7.00089158e+00]
 [ 9.00333104e+00]
 [ 2.99604949e+00]
 [ 5.00103488e+00]
 [ 1.99760518e+00]
 [ 4.00065724e+00]]
Is close verificaion: 	[[False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.49999984]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[7.97809674e-05]
 [1.11781764e-05]
 [6.84615282e-05]
 [8.58403814e-05]
 [8.31222080

In [14]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED APPROXIMATED GRADIENT FOR PROBLEM 1")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q, activate_alt_gradient=True)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)
print("============================================================================")
print("VERSUS NORMAL GRADIENT FOR PROBLEM 1")
print("============================================================================")
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q, activate_alt_gradient=False)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED APPROXIMATED GRADIENT FOR PROBLEM 1
performed 27711 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[ 4.99980495e+00]
 [-1.38336052e-04]
 [ 2.99993888e+00]
 [ 3.00031063e+00]
 [ 7.00007020e+00]
 [ 9.00026301e+00]
 [ 2.99968776e+00]
 [ 5.00008159e+00]
 [ 1.99981051e+00]
 [ 4.00005175e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[-4.17875214e-06]
 [-1.41181957e-06]
 [-2.79459368e-06]
 [-6.23183155e-07]
 [-2.31845624e-06]
 [-7.66840486e-07]
 [-2.59244639e-06]
 [-1.53739595e-06

In [15]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED APPROXIMATED GRADIENT AND SCALING FOR PROBLEM 1")
print("============================================================================")
init=np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]])
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q, activate_alt_gradient=True, scaling = 1000, activate_pre_scaling=True)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)
print("============================================================================")
print("VERSUS NORMAL GRADIENT FOR PROBLEM 1")
print("============================================================================")
x_hat=steepest_descent_multi(init, problem_1_q, grad_problem_1_q, activate_alt_gradient=False)
final_printout(init, minimizer_1, x_hat, problem_1_q, grad_problem_1_q, {}, 1e-3)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED APPROXIMATED GRADIENT AND SCALING FOR PROBLEM 1
performed 27538 iterations. Final norm of gradient: 7.766602168140914e-06
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[5]
 [0]
 [3]
 [3]
 [7]
 [9]
 [3]
 [5]
 [2]
 [4]]
Approximated x is :	[[ 4.99979506e+00]
 [-1.45437433e-04]
 [ 2.99993587e+00]
 [ 3.00032679e+00]
 [ 7.00007398e+00]
 [ 9.00027670e+00]
 [ 2.99967170e+00]
 [ 5.00008591e+00]
 [ 1.99980089e+00]
 [ 4.00005453e+00]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3239.5]]
Function value in approximated point:   [[-3239.5]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[ 2.06790151e-06]
 [-7.05327352e-08]
 [ 2.11876250e-06]
 [ 3.90935895e-06]
 [ 3.03987190e-0

<i>Here is some place for your analysis. How the behavour of algorithm changed after adjustments? What are specific details, differences you noticed with respect to other algorithms behaviour.</i>

### ANALYSIS:

#### Stopping Criterion:

Our Steepest Algorithm is able to find all global minimizers, which is already really good. In some cases the algorithm is not able to achieve an error rate below 0.001, but this happens not often. <br>We could easily improve this by lowering the epsilon (tolerance), but this is also increases the runtime of the algorithm, therefore we decided to take the middle route and accept some "higher" errors. The errors are still below the min requirement of 0.01/0.001. <br><br>
Additionally we tested our <b>advanced</b> stopping criterion, which yields major imporvements for the runtime. We were able to reduce the number of iterations by about 10k.<br>
This is really helpful, if we want to increase the speed of our algorithm. Everything has its price, in this case we loose some accuracy and the error increases.<br> We still have a reasonable low error, but one has to be aware of that.

#### Approximated Gradient:

As we can see above, the approxmation algorithm works perfectly. There is only a small difference between the results. If the difference would be larger, <br>
we would notice that the algorithm converges to infinity and would be able to find the minimizer. Therefore it is really important to find the right tolerance.<br>
Again as already mentioned in the stability section, we have to find the middle way, between low error and fast execution.

#### Scaling:

In our initial version we used a stable learning rate á la NN gradient descent, but changed it to a simple backtracking line search, which improved the runtime by a big margin.<br>
It is hard to tell whether scaling improved stability or something like that, but it apparently cost us a few iterations on our example if we scale down (what we should do according to literature).<br>
This could have something to do with the chosen parameters in backtracking line search. It is overall hard to tell if there are improvements. The reason for that are probably our problems. <br>
Our problems are not really problematic, in regards to scaling and therefore we do not experience big differences, but it could help with difficult examples.


<h3 style="background-color: #D3D92B;"><br>Testing on functions of 1-2 variables, Non-quadratic objective<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Implement functions to optimise over<br></h4><br>
<i>Place for additional comments and argumentation<i>

In [16]:
#your code goes here
# Stationary points of each problem. Number at the end of abc_X is the problem number. abc indicates the name of the of the 3 stationary points a, b and c.
abc_6 = np.array([1, 20, 300])
abc_7 = np.array([4, 50, 600])
abc_8 = np.array([7, 80, 900])
abc_9 = np.array([1, 100, 200])
abc_10 = np.array([2, 120, 200])


# Function to find the global minimizer from all stationary points, which are given by the function creation
def get_global_minimizer(problem, stat_points):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    problem: function which takes x coordinates to calculate y axis values
    --------------------------------------------------------------------------------------------------------------
    stat_points: array of stationary points, which should be compared
    """
    y = []
    for stat_point in np.array(stat_points).astype(float):
        y.append([problem(stat_point), stat_point])
    y = np.array(y)
    return y[y[:,0].argsort()][0, 1]

#############################################################################################################################################################
# problem_x(x) --> is the problem function f, which is used by algorithm.
# derivative_problem_X(x) --> as the name implies, this function is the derivative of the problem function f (problem_X)
# second_derivative_problem_X(x) --> as the name implies, this function is the second derivative of the problem function f (problem_X)
# in all cases you can parse x coordinates to the functions in order to receive the corresponding y axis values.
#############################################################################################################################################################
# Problem 6

def problem_6(x):
    return -(x*(3*x**3+(-4*abc_6[2]-4*abc_6[1]-4*abc_6[0])*x**2+((6*abc_6[1]+6*abc_6[0])*abc_6[2]+6*abc_6[0]*abc_6[1])*x-12*abc_6[0]*abc_6[1]*abc_6[2]))/12

def derivative_problem_6(x):
    return (abc_6[0]-x)*(abc_6[1]-x)*(abc_6[2]-x)

def second_derivative_problem_6(x):
    return -(abc_6[1]-x)*(abc_6[2]-x)-(abc_6[0]-x)*(abc_6[2]-x)-(abc_6[0]-x)*(abc_6[1]-x)
#############################################################################################################################################################
# Problem 7

def problem_7(x):
    return -(x*(3*x**3+(-4*abc_7[2]-4*abc_7[1]-4*abc_7[0])*x**2+((6*abc_7[1]+6*abc_7[0])*abc_7[2]+6*abc_7[0]*abc_7[1])*x-12*abc_7[0]*abc_7[1]*abc_7[2]))/12

def derivative_problem_7(x):
    return (abc_7[0]-x)*(abc_7[1]-x)*(abc_7[2]-x)

def second_derivative_problem_7(x):
    return -(abc_7[1]-x)*(abc_7[2]-x)-(abc_7[0]-x)*(abc_7[2]-x)-(abc_7[0]-x)*(abc_7[1]-x)

#############################################################################################################################################################
# Problem 8

def problem_8(x):
    return -(x*(3*x**3+(-4*abc_8[2]-4*abc_8[1]-4*abc_8[0])*x**2+((6*abc_8[1]+6*abc_8[0])*abc_8[2]+6*abc_8[0]*abc_8[1])*x-12*abc_8[0]*abc_8[1]*abc_8[2]))/12

def derivative_problem_8(x):
    return (abc_8[0]-x)*(abc_8[1]-x)*(abc_8[2]-x)

def second_derivative_problem_8(x):
    return -(abc_8[1]-x)*(abc_8[2]-x)-(abc_8[0]-x)*(abc_8[2]-x)-(abc_8[0]-x)*(abc_8[1]-x)

#############################################################################################################################################################
# Problem 9

def problem_9(x):
    return -(x*(3*x**3+(-4*abc_9[2]-4*abc_9[1]-4*abc_9[0])*x**2+((6*abc_9[1]+6*abc_9[0])*abc_9[2]+6*abc_9[0]*abc_9[1])*x-12*abc_9[0]*abc_9[1]*abc_9[2]))/12

def derivative_problem_9(x):
    return (abc_9[0]-x)*(abc_9[1]-x)*(abc_9[2]-x)

def second_derivative_problem_9(x):
    return -(abc_9[1]-x)*(abc_9[2]-x)-(abc_9[0]-x)*(abc_9[2]-x)-(abc_9[0]-x)*(abc_9[1]-x)

#############################################################################################################################################################
# Problem 10

def problem_10(x):
    return -(x*(3*x**3+(-4*abc_10[2]-4*abc_10[1]-4*abc_10[0])*x**2+((6*abc_10[1]+6*abc_10[0])*abc_10[2]+6*abc_10[0]*abc_10[1])*x-12*abc_10[0]*abc_10[1]*abc_10[2]))/12

def derivative_problem_10(x):
    return (abc_10[0]-x)*(abc_10[1]-x)*(abc_10[2]-x)

def second_derivative_problem_10(x):
    return -(abc_10[1]-x)*(abc_10[2]-x)-(abc_10[0]-x)*(abc_10[2]-x)-(abc_10[0]-x)*(abc_10[1]-x)

<hr><h4 style="background-color: #ADB8FF;"><br>Run 5 tests<br></h4><br>
<p><i>Place for your additional comments and argumentation<i>

In [28]:
#your code goes here
x_start = np.array([[100]], dtype=float)  # start point for all problems
print("============================================================================")
print("DEFAULT TESTS WITHOUT ANY ADDITIONAL FEATURES")
print("============================================================================")
print("**** PROBLEM 6 ****")
x_hat = steepest_descent_uni(x_start, problem_6,  derivative_problem_6)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)
print("============================================================================")
print("**** PROBLEM 7 ****")
x_hat=steepest_descent_uni(x_start, problem_7,  derivative_problem_7)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_7, abc_7), x_appr=x_hat, f=problem_7, grad=derivative_problem_7, args={}, tolerance=1e-4)
print("============================================================================")
print("**** PROBLEM 8 ****")
x_hat=steepest_descent_uni(x_start, problem_8,  derivative_problem_8)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_8, abc_8), x_appr=x_hat, f=problem_8, grad=derivative_problem_8, args={}, tolerance=1e-4)
print("============================================================================")
print("**** PROBLEM 9 ****")
x_start = np.array([[90]], dtype=float)  # start point for all problems
x_hat=steepest_descent_uni(x_start, problem_9,  derivative_problem_9)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_9, abc_9), x_appr=x_hat, f=problem_9, grad=derivative_problem_9, args={}, tolerance=1e-4)
print("============================================================================")
print("**** PROBLEM 10 ****")
x_start = np.array([[100]], dtype=float)  # start point for all problems
x_hat=steepest_descent_uni(x_start, problem_10,  derivative_problem_10)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_10, abc_10), x_appr=x_hat, f=problem_10, grad=derivative_problem_10, args={}, tolerance=1e-4)

DEFAULT TESTS WITHOUT ANY ADDITIONAL FEATURES
**** PROBLEM 6 ****
performed 43127 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[9.99872796e-06]]

Is close verificaion:
[[ True]]
**** PROBLEM 7 ****
performed 9827 iterations.
Initial x is :		[[100.]]
Optimal x is :		50.0
Approximated x is :	[[50.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-9062500.0
Function value in approximated point:   [[-9062500.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[9.99020884e-06]]

Is close verificaion:
[[ True]]
**** PROBLEM 8 ****
performed 4213 iterations.
Initial x is :		[[100.]]
Optimal x

In [18]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED ADVANCED STOPPING CRITERION FOR PROBLEM 6")
print("============================================================================")
x_start = np.array([[100]], dtype=float)  # start point for all problems
x_hat = steepest_descent_uni(x_start, problem_6,  derivative_problem_6, activate_new_stop_crit=True)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)
print("============================================================================")
print("VERSUS NORMAL STOPPING CRITERION FOR PROBLEM 6")
print("============================================================================")
x_hat = steepest_descent_uni(x_start, problem_6,  derivative_problem_6, activate_new_stop_crit=False)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED ADVANCED STOPPING CRITERION FOR PROBLEM 6
*** Y DIFFERENCE SMALLER THAN ADJUSTED TOLERANCE ***
performed 20629 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.00029749]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-327999.99976459]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[1.58265798]]

Is close verificaion:
[[False]]
VERSUS NORMAL STOPPING CRITERION FOR PROBLEM 6
performed 43127 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[9.99872796e-0

In [19]:
print("============================================================================")
print("ADDITIONAL TESTS WITH ACTIAVTED FEATURES")
print("============================================================================")
print("ACTIVATED APPROXIMATED GRADIENT FOR PROBLEM 6")
print("============================================================================")
x_start = np.array([[100]], dtype=float)  # start point for all problems
x_hat = steepest_descent_uni(x_start, problem_6,  derivative_problem_6, activate_alt_gradient=True)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)
print("============================================================================")
print("VERSUS NORMAL GRADIENT FOR PROBLEM 6")
print("============================================================================")
x_hat = steepest_descent_uni(x_start, problem_6,  derivative_problem_6, activate_alt_gradient=False)
final_printout(x_0=x_start, x_optimal=get_global_minimizer(problem_6, abc_6), x_appr=x_hat, f=problem_6, grad=derivative_problem_6, args={}, tolerance=1e-4)

ADDITIONAL TESTS WITH ACTIAVTED FEATURES
ACTIVATED APPROXIMATED GRADIENT FOR PROBLEM 6
performed 42912 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[1.03201677e-05]]

Is close verificaion:
[[ True]]
VERSUS NORMAL GRADIENT FOR PROBLEM 6
performed 43127 iterations.
Initial x is :		[[100.]]
Optimal x is :		20.0
Approximated x is :	[[20.]]
Is close verificaion: 	[[ True]]

Function value in optimal point:	-328000.0
Function value in approximated point:   [[-328000.]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
-0.0

Gradient approximation in approximated point is:
[[9.99872796e-06]]

Is close verificaion:
[[ True]]


<i>Here is some place for your analysis. How the behavour of algorithm changed after adjustments? What are specific details, differences you noticed with respect to other algorithms behaviour.</i>

### ANALYSIS:

#### Stopping Criterion:

Our Steepest Algorithm is able to find all global minimizers, which is already really good. In some cases the algorithm is not able to achieve an error rate below 0.001, but this happens not often. <br>We could easily improve this by lowering the epsilon (tolerance), but this is also increases the runtime of the algorithm, therefore we decided to take the middle route and accept some "higher" errors. The errors are still below the min requirement of 0.01/0.001. <br><br>
Additionally we tested our <b>advanced</b> stopping criterion, which yields major imporvements for the runtime. We were able to reduce the number of iterations by about 10k.<br>
This is really helpful, if we want to increase the speed of our algorithm. Everything has its price, in this case we loose some accuracy and the error increases.<br> We still have a reasonable low error, but one has to be aware of that.

#### Approximated Gradient:

As we can see above, the approxmation algorithm works perfectly. There is only a small difference between the results. If the difference would be larger, <br>
we would notice that the algorithm converges to infinity and would be able to find the minimizer. Therefore it is really important to find the right tolerance.<br>
Again as already mentioned in the stability section, we have to find the middle way, between low error and fast execution.


## Comparison:

#### Steepest Descent (CURRENT):

Needs rescaling for unbalanced problem function.<br>
Needs many iterations >20k to find minimizer.<br>
Does not reach lowest error bound, with advanced stopping criterion. This could be fixed, but runtime would be higher.<br>
Stable as long problems are not too complicated (unbalanced, many local min, etc.)<br>

#### Conjugate Gradient:

Needs rescaling for unbalanced problem function.<br>
Needs many iterations >20k to find minimizer.<br>
Does not reach lowest error bound, with advanced stopping criterion. This could be fixed, but runtime would be higher. (Better than steepest descent, still not optimal)<br>
Stable as long problems are not too complicated (unbalanced, many local min, etc.)<br>

#### Newton:

Does not need rescaling for unbalanced problem function. Newton default behaviour.<br>
Really fast <10 iterations for our examples to find solution.<br>
Reaches lowest error bound with and without additional features.<br>
Stable with and without additional features, this could change for more difficult problems, but has not been tested <br>

#### Quasi Netwon:


Does not need rescaling for unbalanced problem function. Newton default behaviour.<br>
Really fast <30 iterations for our examples to find solution.<br>
Reaches lowest error bound with and without additional features.<br>
Stable with and without additional features, this could change for more difficult problems, but has not been tested <br>

<h3 style="background-color: #D3D92B;"><br>Template for teachers' tests<br></h3><br>
<hr><h4 style="background-color: #ADB8FF;"><br>Set up a template, how one can run your code<br></h4><br>
Template should include sceletons for:<ul>
    <li>custom function to optimise over </li> 
    <li>values initialisation to submit into otimising algorithm </li> 
    <li>optimiser function call</li> 
    <li>report print out call</li> </ul><br>
Provide descriptions and comments.

In [39]:
"""
**** Teacher Test ****
"""

"""
** Custom Function to optimise over
------------------------------------------------------------------------------------------------------------------
- All problems, quadratic and non quadratic are based on the instruction from project phase 1.
*** Quadratic problems
------------------------------------------------------------------------------------------------------------------
- One can easily create new quadratic problems by changing the seed of the "get_random_A_b_minimizer" function. This function uses the seed to create random matrices, which are than used for the problem function.
- The three functions problem_x_q, grad_problem_x and hessian_problem_x are used by the optimizer and those functions use the return values of the get_random_A_b_minimizer functions to create the problem function.
- Therefore one can change the seed in the mentioned function and create a new problem, which can be recreated and is still random.
- It is also possible to create completly different problems, but the structure has to match.
------------------------------------------------------------------------------------------------------------------
*** Non Quadratic problems
------------------------------------------------------------------------------------------------------------------
- One can also create new non quadratic problems by changing the values of the "abc_x" array. Those are the pre defined stationary points of the problem function. By choosing different points, the problem changes.
- The three functions problem_x, derivative_problem_x and second_derivative_problem_x are used by the optimizer and those functions use the abc_x array to create the problem function.
- Therefore one can change the content of the abc_x array and create a new problem. 
- !!!!!!!!!!!!!!!IMPORTANT!!!!!!!!!!!!!!!!!!!!: No additional values should be entered, this means the length of this array should be 3, otherwise the solution might not match.
- It is also possible to create completly different problems, but the structure has to match.
"""

def get_random_A_b_minimizer(seed):
    """
    Parameters
    --------------------------------------------------------------------------------------------------------------
    seed: Seed is an int value, which is used to create reproducable random values.
    --------------------------------------------------------------------------------------------------------------
    - 10x10 random matrix is produced by this function, which is the base of the problem
    - 10x1 random vector is produced, which is also the minimizer of the function and the goal of the algo
    - Function uses thoes variables to create a problem function, which can be solved by the algo
    """
    np.random.seed(seed)
    A=np.random.randint(0, 2, (10, 10))
    A=A@A.T

    np.random.seed(seed)
    minimizer=np.random.randint(0, 10, (10, 1))

    b=A@minimizer

    return A, b, minimizer

# Quadratic problem
A_6, b_6, minimizer_6 = get_random_A_b_minimizer(99)

def problem_6_q(x):
    return (x.T@A_6@x)/2-b_6.T@x

def grad_problem_6_q(x):
    return np.dot(A_6,x)-b_6

def hessian_problem_6_q(x):
    return A_6

#############################################################################################################################################################
# Non quadratic problem

abc_11 = np.array([2, 200, 400])
def problem_11(x):
    return -(x*(3*x**3+(-4*abc_11[2]-4*abc_11[1]-4*abc_11[0])*x**2+((6*abc_11[1]+6*abc_11[0])*abc_11[2]+6*abc_11[0]*abc_11[1])*x-12*abc_11[0]*abc_11[1]*abc_11[2]))/12

def derivative_problem_11(x):
    return (abc_11[0]-x)*(abc_11[1]-x)*(abc_11[2]-x)

def second_derivative_problem_11(x):
    return -(abc_11[1]-x)*(abc_11[2]-x)-(abc_11[0]-x)*(abc_11[2]-x)-(abc_11[0]-x)*(abc_11[1]-x)

"""
--------------------------------------------------------------------------------------------------------------
** Value Initalisation Optimizer & Report Call 
--------------------------------------------------------------------------------------------------------------
- Call function to optimise over and safe result
- Optimizer function is called with "steppest_descent_multi" for quadratic problems and "steppest_descent_uni" for non quadratic.
- The names come from the number of variables used. There is a strict differnce in our functions between gradient and derivative. It would be possible to use the same function for both problems, but 
- kept it consistent to the individual task
- Decide on a example. 
-- There are 5 quadratic problems named problem_X_q (q for quadratic), the gradient is than named grad_problem_X_q and same for the hessian (X == Number of problem 1..5)
-- There are also 5 non quadratic problems named problem_X, the derivative is than named derivative_problem_X and same for the second derivative (X == Number of problem 6..10)
- We have to also define a starting point for the optimizer. All of our problems, in the quadratic case, have to in shape 10x1, since our quadratic problems have 10 variables
- The non quadratic problems need a 1x1 starting point, since our problems have only 1 variable.
- Additionally one has to choose the extra features. For Steepest descent we implemented the advanced stopping criterion and the approximated gradient and scaling (only for multi). Inverse is not implemented, for reasons checkout the corresponding chapters.
- For the final printout, we use the given function. There is a difference between quadratic and non quadratic problems. Our non quadratic problems have 3 stationary points and we have to call the function
  get_global_minimizer to find the real minimizer. This has to be done for the non quadratic problems.
- x_optimal is for the quadratic problem the variable called "minimizer_X", where X has to be replaced by the problem number (1..5)
- For the non quadratic problems we have an array called "abc_X", where X has to be replaced by the problem number (6..10), which contains all stationary points.
"""
##########################################################################################################################################################################################
# Quadratic problem call

x_start = np.array([[6],[1],[2],[6],[7],[8],[1],[5],[1],[3]]) # 10x1 start point for optimizer
problem = problem_6_q # Choose 3rd quadratic problem as problem to be solved by the optimizer
gradient_problem = grad_problem_6_q # Choose 3rd quadratic problem gradient to be solved by the optimizer
x_optimal = minimizer_6
# We could choose different problems and problem gradients, but that does not make sense, therefore it is not recommended :).
alt_gradient_flag = True # Here one can activate (True), the approximated gradient or deactivate (False) it.
alt_stopping_crit = True # Here one can activate (True), the advanced stopping criterion or deactivate (False) it.
activate_pre_scaling = True # Here one can activate (True), the pre scaling or deactivate (False) it.
scaling = 1000 # Here on can choose the caling value, which should be used by algorithm

# Optimizer Function Call. All the above defined options are parsed to the optimizer call
x_hat = steepest_descent_multi(x_start, problem,  gradient_problem, activate_alt_gradient=alt_gradient_flag, activate_new_stop_crit=alt_stopping_crit, activate_pre_scaling=activate_pre_scaling, scaling=scaling)
# Final Printout uses the result of the optimizer + the given statements and verifys the results. Args is not used, since our algorithm and functions do not support it.
final_printout(x_0=x_start, x_optimal=x_optimal, x_appr=x_hat, f=problem, grad=gradient_problem, args={}, tolerance=1e-2)
print("==================================================================================================================================")
##########################################################################################################################################################################################
# Non Quadratic problem call

x_start = np.array([[100]], dtype=float) # 1x1 start point for optimizer
problem = problem_11 # Choose 6th non quadratic problem as problem to be solved by the optimizer
gradient_problem = derivative_problem_11 # Choose 6th non quadratic problem derivaitve to be solved by the optimizer
x_optimal = get_global_minimizer(problem_11, abc_11)
# We could choose different problems and problem gradients, but that does not make sense, therefore it is not recommended :).
alt_gradient_flag = False # Here one can activate (True), the approximated gradient or deactivate (False) it.
alt_stopping_crit = True # Here one can activate (True), the advanced stopping criterion or deactivate (False) it.

x_hat = steepest_descent_uni(x_start, problem,  gradient_problem, activate_alt_gradient=alt_gradient_flag, activate_new_stop_crit=alt_stopping_crit)
final_printout(x_0=x_start, x_optimal=x_optimal, x_appr=x_hat, f=problem, grad=gradient_problem, args={}, tolerance=1e-3)

*** Y DIFFERENCE SMALLER THAN ADJUSTED TOLERANCE ***
performed 2754 iterations.
Initial x is :		[[6]
 [1]
 [2]
 [6]
 [7]
 [8]
 [1]
 [5]
 [1]
 [3]]
Optimal x is :		[[1]
 [3]
 [9]
 [8]
 [9]
 [8]
 [2]
 [4]
 [5]
 [4]]
Approximated x is :	[[1.00192138]
 [3.00344123]
 [8.99699249]
 [7.99813051]
 [8.99548132]
 [7.99661354]
 [1.99861108]
 [4.00166592]
 [5.00208357]
 [4.00423351]]
Is close verificaion: 	[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]

Function value in optimal point:	[[-3911.5]]
Function value in approximated point:   [[-3911.49999833]]
Is close verificaion:	[[ True]]

Gradient approximation in optimal point is:
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

Gradient approximation in approximated point is:
[[0.0006771 ]
 [0.00101023]
 [0.00010752]
 [0.00073349]
 [0.00042642]
 [0.00037289]
 [0.00083196]
 [0.0007534 ]
 [0.00071559]
 [0.00043687]]

Is close verificaion:
[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 